<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Data_collection_Country_P17_P27_P495.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import of the training data

from google.colab import files
uploaded = files.upload()

# Import of moduls

!pip install wikidata --quiet

import matplotlib.pyplot as plt
import pandas as pd
from wikidata.client import Client

from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading the data
train_df = pd.read_csv('[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep='\t')

Saving [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv to [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv


In [2]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

In [3]:
def get_claims(wikidata_url):
  # Extract the id of the item
    entity_id = extract_entity_id(wikidata_url)

    # Extract information of the item
    client = Client()
    item = client.get(entity_id, load=True)

    # Extract the claims of the item
    claims = item.data.get("claims", {})

    return claims

In [4]:
def get_country(wikidata_url):

    claims = get_claims(wikidata_url)

    country_of_origin_id = None
    country_id = None
    nationality_id = None

    # Extract the statement "P495" : Country of origin
    unicode_statements = claims.get("P495", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})

            country_of_origin_id = value.get("id")

    # Extract the statement "P17" : Country
    unicode_statements = claims.get("P17", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})

            country_id = value.get("id")

    # Extract the statement "P27" : Nationality
    unicode_statements = claims.get("P27", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})

            nationality_id = value.get("id")

    return country_of_origin_id, country_id, nationality_id


# Test on one line of the DataFrame
#wikidata_url = train_df.iloc[532, 0]
#get_country(wikidata_url)

In [5]:
# We merge the variables, in order to have only one.

def merge_variables(df, variables):
  """
  Take in input, the variables of a dataframe, and return a dataframe with a single variable => the first value founded across the rows
  """
  return df[variables].bfill(axis=1).iloc[:, 0]

In [6]:
# Apply the function to a subset of the DataFrame
subset = range(30,40)
country_informations = train_df.loc[subset, "item"].apply(lambda url: get_country(url))

variables = ['country_of_origin_id', 'country_id', 'nationality_id']
country_informations_df = pd.DataFrame(country_informations.tolist(), columns=variables)

# We merge the variables, in order to have only one
country_informations_df['country'] = merge_variables(country_informations_df, variables)
country_informations_df.drop(variables, axis=1, inplace=True)

# We add the new variable to the training dataset
train_df['country'] = country_informations_df['country']

# Visualisation
train_df.head()

,item,name,description,type,category,subcategory,label,country
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,Q8733
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,Q145
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,Q928
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,Q30
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,Q1747689


In [ ]:
# When we want to have all the variables, instead of merging them in only one variable

#subset = range(530,540)
#country_informations = train_df.loc[subset, "item"].apply(lambda url: get_country(url))

#train_df.loc[subset, "country_of_origin_id"] = country_informations.apply(lambda x: x[0])
#train_df.loc[subset, "country_id"] = country_informations.apply(lambda x: x[1])
#train_df.loc[subset, "nationality_id"] = country_informations.apply(lambda x: x[2])

In [ ]:
train_df.iloc[subset]

,item,name,description,type,category,subcategory,label,country_of_origin_id,country_id,nationality_id
530,http://www.wikidata.org/entity/Q695599,Austrian Federal Government,executive cabinet of the Republic of Austria,concept,politics,government,cultural exclusive,None,Q40,None
531,http://www.wikidata.org/entity/Q837416,coalition government,government consisting of two or more parties,concept,politics,government,cultural agnostic,None,None,None
532,http://www.wikidata.org/entity/Q951187,Mekitsa,food,concept,food,food,cultural exclusive,Q219,Q219,None
533,http://www.wikidata.org/entity/Q943696,thenthuk,food,concept,food,food,cultural exclusive,Q17252,None,None
534,http://www.wikidata.org/entity/Q964296,Kulcha,food,concept,food,food,cultural representative,Q843,None,None
535,http://www.wikidata.org/entity/Q937013,bouillon cube,dehydrated broth or stock formed into a small ...,concept,food,food,cultural agnostic,None,None,None
536,http://www.wikidata.org/entity/Q957696,grit,bran and chaff of cereal grains,concept,food,food,cultural agnostic,None,None,None
537,http://www.wikidata.org/entity/Q936218,Chakhchoukha,تركيا,concept,food,food,cultural exclusive,Q262,Q262,None
538,http://www.wikidata.org/entity/Q2247208,Schneeball,German dessert,concept,food,food,cultural exclusive,Q183,Q183,None
539,http://www.wikidata.org/entity/Q2308904,Malai,desert from India,concept,food,food,cultural representative,None,None,None
